In [1]:
# Importación de librerias.
import pandas as pd
from gensim.models import KeyedVectors
import numpy as np
import matplotlib.pyplot as plt

# Carga del dataset. 
analogias_df = pd.read_csv('analogies_answers.csv')
analogias_df.head()

,cueEjemplo,targetEjemplo,cue,response,count,rank,porcentaje,ignorar
0,bailarín,bailar,albañil,construir,1,1.0,100.0,False
1,pistola,bala,arco,flecha,1,1.0,100.0,False
2,picnic,parque,asado,parrillero,1,1.0,100.0,False
3,pirámide,momia,ataúd,cadáver,1,1.0,100.0,False
4,tren,vías,auto,carretera,1,1.0,100.0,False


In [2]:
# Embeddings utilizados.
embeddings_WIKI = KeyedVectors.load_word2vec_format('embeddings/wiki.es.vec', limit=None)
embeddings_SUC = KeyedVectors.load_word2vec_format('embeddings/embeddings-l-model.vec', limit=None)
embeddings_SWOW = KeyedVectors.load_word2vec_format('embeddings/swow.embedding.was.26-04-2022.vec', limit=None)
lower_embeddings_SWOW = KeyedVectors(vector_size=embeddings_SWOW.vector_size)
embeddings_SBWC = KeyedVectors.load_word2vec_format('embeddings/SBW-vectors-300-min5.txt', limit=None)
embeddings_Glove = KeyedVectors.load_word2vec_format('embeddings/glove-sbwc.i25.vec', limit=None)

In [3]:
# Embedding SWOW pasado a minusculas 

# Vectores y palabras del SWOW.
palabras = list(embeddings_SWOW.key_to_index.keys())
vectores = list(embeddings_SWOW.vectors)

# Convertir todas las palabras a minúsculas.
palabras_minusculas = [palabra.lower() for palabra in palabras]

# Creación de un nuevo modelo SWOW con las palabras y vectores en minúsculas. 
lower_embeddings_SWOW = KeyedVectors(vector_size=embeddings_SWOW.vector_size)
lower_embeddings_SWOW.add_vectors(palabras_minusculas, vectores)

In [4]:
# Listado de los embeddings utilizados. 
lista_de_Embeddings = [embeddings_WIKI, embeddings_SUC, lower_embeddings_SWOW, embeddings_SBWC, embeddings_Glove]
#lista_de_Embeddings = [embeddings_WIKI, embeddings_SUC, lower_embeddings_SWOW, embeddings_SBWC, embeddings_Glove

In [5]:
# Calculo de la frecuencia de las combinaciones de 'cue' y 'response' y sus rangos para identificar las analogías más relevantes
analogias_df['count'] = analogias_df.groupby(['cue','response'])['response'].transform('count')
analogias_df['rank'] = analogias_df.groupby('cue')['count'].rank(ascending=False, method='min')

# Calculo del porcentaje de frecuencia de cada combinación en relación con el número total de respuestas para ese 'cue'.
total_respuestas = analogias_df.groupby('cue')['response'].transform('count')
analogias_df['porcentaje'] = (analogias_df['count'] / total_respuestas) * 100

# Indices de filas que tienen rango minimo (rango 1).
idx = analogias_df.groupby(['cue'])['rank'].idxmin()

# Me quedo unicamente con las filas que tienen las combinaciones más relevantes según el rango.
analogias_df = analogias_df.loc[idx]

# Se guarda el DataFrame modificado en un archivo CSV.
analogias_df.to_csv('analogies_answers.csv', index=False)

# Se pasan todas las palabras a minúsculas. 
analogias_df['cueEjemplo'] = analogias_df['cueEjemplo'].apply(lambda x: str(x).lower())
analogias_df['targetEjemplo'] = analogias_df['targetEjemplo'].apply(lambda x: str(x).lower())
analogias_df['cue'] = analogias_df['cue'].apply(lambda x: str(x).lower())

# Se realizan los reemplazos necesarios. 
analogias_df['cueEjemplo'] = analogias_df['cueEjemplo'].str.replace('lápiz labial', 'labial').replace('estados unidos', 'eeuu').replace('buenos aires', 'bsas')
analogias_df['targetEjemplo'] = analogias_df['targetEjemplo'].str.replace('lápiz labial', 'labial').replace('estados unidos', 'eeuu').replace('buenos aires', 'bsas')
analogias_df['cue'] = analogias_df['cue'].str.replace('lápiz labial', 'labial').replace('estados unidos', 'eeuu').replace('buenos aires', 'bsas')


In [6]:
from mis_funciones import metricas_topK, calcular_predicciones

# Funcion calcular predicciones. 
K = 30
todas_predicciones = [calcular_predicciones(e, analogias_df, K) for e in lista_de_Embeddings]

# Funcion calcular metricas topK.
topK_metricas_todas = [metricas_topK (prediccion, analogias_df) for prediccion in todas_predicciones]
print("Todas las metricas topK: ", topK_metricas_todas)

Una de las palabras no está en el embedding: "Key 'momia' not present in vocabulary"
Una de las palabras no está en el embedding: "Key 'bsas' not present in vocabulary"
Una de las palabras no está en el embedding: "Key 'peluquera' not present in vocabulary"
Una de las palabras no está en el embedding: "Key 'samba' not present in vocabulary"
Una de las palabras no está en el embedding: "Key 'eeuu' not present in vocabulary"
Una de las palabras no está en el embedding: "Key 'judaísmo' not present in vocabulary"
Una de las palabras no está en el embedding: "Key 'manubrio' not present in vocabulary"
Una de las palabras no está en el embedding: "Key 'eeuu' not present in vocabulary"
Una de las palabras no está en el embedding: "Key 'tereré' not present in vocabulary"
Una de las palabras no está en el embedding: "Key 'bsas' not present in vocabulary"
Todas las metricas topK:  [0.3108108108108108, 0.44594594594594594, 0.5, 0.3108108108108108, 0.4594594594594595]
